In [1]:
import os
import shutil
import calendar
import pandas as pd

In [2]:
base = "bucket/original_dataset_customs"
out_dir = "dataset_customs"

In [3]:
im_ex_dir = [os.path.join(base,x) for x in os.listdir(base) if os.path.isdir(os.path.join(base,x))]
im_ex_dir

['bucket/original_dataset_customs/export',
 'bucket/original_dataset_customs/import']

# Count files

In [4]:
datas = []
for category in os.listdir(base):
    path1 = os.path.join(base, category)
    if os.path.isdir(path1):
        for hs_code in os.listdir(path1):
            path2 = os.path.join(path1, hs_code)
            if os.path.isdir(path2):
                datas.append({
                    'type':category,
                    'hs_code':hs_code,
                    'num_file': len(os.listdir(path2))
                })
df = pd.DataFrame(datas)
df

,type,hs_code,num_file
0,export,4204,252
1,export,8453,252
2,export,8462,252
3,export,8001,252
4,export,2917,252
...,...,...,...
2502,import,3817,252
2503,import,1806,252
2504,import,4602,252
2505,import,8473,252


In [5]:
df_pivot = pd.pivot_table(df, columns=['type','hs_code']).T
print("Total files : {:,}".format(df_pivot['num_file'].sum()))
df_pivot

Total files : 631,733


num_file
type   hs_code          
export 0101          252
       0102          252
       0103          252
       0104          252
       0105          252
...                  ...
import 9702          252
       9703          252
       9704          252
       9705          252
       9706          252

[2507 rows x 1 columns]

# Define function for convert file name

In [6]:
months = [elm for elm in calendar.month_name]
def month2int(month):
    return months.index(month)

def convert2com(f_name):
    month_name, year = f_name.split(' ')[-2:]
    year = year.replace('.csv','')
    month = month2int(month_name)
    return "{}-{:02d}.csv".format(year, month)

# Main Loop : Convert file names

In [8]:
for data_type in im_ex_dir:
    category_paths = [os.path.join(data_type, x) for x in os.listdir(data_type) if os.path.isdir(os.path.join(data_type, x))]
    category_paths.sort()
    for category_path in category_paths:
        new_category_path = category_path.split('/')
        new_category_path[1] = out_dir
        new_category_path = "/".join(new_category_path)
        if not os.path.exists(new_category_path):
            os.makedirs(new_category_path)
        files_path = [os.path.join(category_path, x) for x in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, x))]
        for file_path in files_path:
            f_paths = file_path.split('/')
            new_f_name = convert2com(f_paths[-1])
            f_paths[-1] = new_f_name
            f_paths[1] = out_dir
            file_path_new = "/".join(f_paths)
            shutil.copyfile(file_path, file_path_new)